# **<center><font style="color:rgb(100,109,254)">Module 7: Physics + Computer Vision</font> </center>**

<center>
    <img src='https://drive.google.com/uc?export=download&id=1fZcQOIeX7OVJhsmMH5COSOzbBAxA28UG' width=800> 
</center>
    

## **<font style="color:rgb(134,19,348)"> Module Outline </font>**

The module can be split into the following parts:

- *Lesson 1: Introduction to Pymunk*

- *Lesson 2: Integrating Pymunk with OpenCV*

- ***Lesson 3:* Build the Final Application** *(This Tutorial)*

**Please Note**, these Jupyter Notebooks are not for sharing; do read the Copyright message below the Code License Agreement section which is in the last cell of this notebook.
-Taha Anwar

Alright, let's get started.

### **<font style="color:rgb(134,19,348)"> Import the Libraries</font>**

As usual, first, we will import the required libraries.

In [1]:
import cv2
import pymunk
import numpy as np
import mediapipe as mp
from time import time
from collections import deque
from previous_lesson import (detectHandsLandmarks, recognizeGestures,
                             calculateDistance, rotate, convertPoints, createBall, 
                             drawBalls, createObstacle, drawObstacles)

## **<font style="color:rgb(134,19,348)">Initialize the Hands Landmarks Detection Model</font>**

After that, as we have been doing in the previous modules, we will have to initialize the **`mp.solutions.hands`** class and then set up the **`mp.solutions.hands.Hands()`** function with appropriate arguments and also initialize **`mp.solutions.drawing_utils`** class that is needed to visualize the detected landmarks. 

In [2]:
# Initialize the mediapipe hands class.
mp_hands = mp.solutions.hands

# Set up the Hands functions for videos.
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, 
                       min_detection_confidence=0.8, min_tracking_confidence=0.8)

# Initialize the mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils

## **<font style="color:rgb(134,19,348)">Create a Function to Make Arbitrary Shaped Objects</font>**

Now we will create a function **`createObject()`** that will create an arbitrarily shaped object by attaching different line segments to a body utilizing Pymunk's [**`Body`**](http://www.pymunk.org/en/latest/pymunk.html#pymunk.Body) class and the [**`Segment`**](http://www.pymunk.org/en/latest/pymunk.html#pymunk.Segment) class.

In [3]:
def createObject(frame, drawing_body, start_point, end_point):
    '''
    This function will create an arbitrary shaped object by attaching different line segments to a body.
    Args:
        frame:        A frame/image from a real-time webcam feed. 
        drawing_body: The body of the arbitrary shaped object, that is to be created.
        start_point:  The starting point coordinates of the line segment, that is to be attached to the body.
        end_point:    The starting point coordinates of the line segment, that is to be attached to the body.
    Returns:
        drawing_body:       The body of the arbitrary shaped object, after being attached to a new line segment.
        drawing_shape:      The line segment shape that is attached to the body.
        segment_start_diff: The line segment starting point w.r.t the body position.
        segment_end_diff:   The line segment ending point w.r.t the body position.
    '''
    
    # Convert the starting and ending points from OpenCV coordinates system to the Pymunk coordinates system. 
    start_x, start_y = convertPoints(frame, start_point)
    end_x, end_y = convertPoints(frame, end_point)
    
    # Check if a drawing body is not created.
    if not drawing_body:
        
        # Create a dynamic body for the object. 
        drawing_body = pymunk.Body(body_type=pymunk.Body.DYNAMIC)
        
        # Set the body's position equal to the starting point.
        drawing_body.position = start_x, start_y
    
    # Get the position of the body to which we have to attach the shape.
    body_x, body_y = drawing_body.position
    
    # Get the starting and ending points w.r.t the body position.
    segment_start_diff =  start_x-body_x, start_y-body_y
    segment_end_diff = end_x-body_x, end_y-body_y
    
    # Create a line segment shape and attach it to the body. 
    drawing_shape = pymunk.Segment(body=drawing_body, a=segment_start_diff, b=segment_end_diff, radius=brush_size)
    
    # Set the density of the shape. 
    drawing_shape.density = 1
    
    # Set the friction of the shape.
    drawing_shape.friction = 1.0
    
    # Return the body, shape, and the starting and ending points w.r.t the body position.
    return drawing_body, drawing_shape, (segment_start_diff, segment_end_diff)

## **<font style="color:rgb(134,19,348)">Create a Function to Draw Objects</font>**

And now we will create a function **`drawObjects()`**, that will utilize the [**`cv2.line()`**](https://docs.opencv.org/4.x/d6/d6e/group__imgproc__draw.html#ga7078a9fae8c7e7d13d24dac2520ae4a2) function to draw all line segments making the arbitrarily shaped objects on a real-time webcam feed.

In [4]:
def drawObjects(frame, objects_to_draw, brush_size):
    '''
    This function draws the arbitrary shaped objects on a frame/image.
    Args:
        frame:           The frame/image to draw objects on, from a real-time webcam feed. 
        objects_to_draw: A list containing the objects body, and the starting and ending points of the line segments w.r.t the body position.
        brush_size:      The thickness of the line segment, that is to be drawn.
    Returns:
        frame: The frame/image with the objects drawn, from a real-time webcam feed.
    '''
    
    # Iterate over the line segments to draw. 
    for segment in objects_to_draw:
        
        # Get the object's body, and the starting and ending points of the line segment w.r.t the body position. 
        body, (segment_start_diff, segment_end_diff) = segment
        
        # Get the body's position.
        body_x, body_y = body.position
        
        # Get the starting and ending points coordinates of the line segment on the frame.
        start_point =  body_x+segment_start_diff[0], body_y+segment_start_diff[1]
        end_point = body_x+segment_end_diff[0], body_y+segment_end_diff[1]
        
        # Rotate the starting and ending points coordinates w.r.t body angle and position being origin.
        start_point = rotate(point=start_point, origin=body.position, angle=body.angle)
        end_point = rotate(point=end_point, origin=body.position, angle=body.angle)
        
        # Draw the line segment on the frame.
        cv2.line(img=frame, pt1=convertPoints(frame, start_point), 
                 pt2=convertPoints(frame, end_point), color=(255,255,255), thickness=brush_size)
        
    # Return the frame.
    return frame

And now we will utilize the functions created above, and in the previous lessons to create balls and arbitrary objects with hand gestures on a real-time webcam feed.

In [7]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)
camera_video.set(3,1280)
camera_video.set(4,960)

# Get the height and width of the webcam video.
height, width = (int(camera_video.get(cv2.CAP_PROP_FRAME_HEIGHT)), 
                 int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH)))

# Create a named window for resizing purposes.
cv2.namedWindow('Webcam Feed', cv2.WINDOW_NORMAL)

# Create a Space which will contain the simulation.
space = pymunk.Space()

# Set the gravity.
space.gravity = 0, -500 

# Initialize a dictionary that will contain the balls, boundary lines, and the objects to draw.
to_draw = {'Balls': [], 'boundary_lines': [], 'Objects':[]}

# Create line segments and append them into the list inside the dictionary.
to_draw['boundary_lines'].append(createObstacle(space, starting_point=(0, 5),
                                         ending_point=(width, 5), thickness=20))
to_draw['boundary_lines'].append(createObstacle(space, starting_point=(10, 0),
                             ending_point=(10, height), thickness=20))
to_draw['boundary_lines'].append(createObstacle(space, starting_point=(width-10, 0),
                             ending_point=(width-10, height), thickness=20))

# Initialize a variable to store the body of the object being created.
drawing_body=None

# Initialize a list to store the shapes of the object being created. 
shapes=[]

# Initialize a variable to store the brush size (i.e., thickness of the lines forming the object).
brush_size = 20

# Initialize variables to store previous x and y location.
# That are hand brush x and y coordinates in the previous frame.
prev_x = None 
prev_y = None

# Initialize a variable to store the buffer length.
BUFFER_MAX_LENGTH = 10

# Initialize a buffer to store recognized gestures.
buffer = deque([], maxlen=BUFFER_MAX_LENGTH)

# Initialize a variable to store the frame count.
frame_count = 0

# Get the start time.
start_time = time()

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
   
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then 
    # continue to the next iteration to read the next frame.
    if not ok:
        continue
        
    # Increment the frame counter.
    frame_count += 1
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Perform Hands landmarks detection on the frame.
    frame, results = detectHandsLandmarks(frame, hands, draw=True, display=False)
    
    # Check if the hands landmarks in the frame are detected.
    if results.multi_hand_landmarks:
        
        # Perform a hand gesture recognition.
        # I have modified this recognizeGestures() function, to return the fingers tips position of the both hands.
        current_gesture, hands_tips_positions = recognizeGestures(frame, results, hand_label='LEFT', draw=False,
                                                                  display=False)
        # Check if a known gesture is recognized.
        if current_gesture != 'UNKNOWN':
            
            # Check if all the gestures stored in the buffer are equal to the current gesture.
            if all(current_gesture==gesture for gesture in buffer):
                
                # Append the current gesture into the buffer.
                buffer.append(current_gesture)
                
            # Otherwise.
            else:
                
                # Clear the buffer.
                buffer.clear()
            
            # Check if the length of the buffer is equal to the maxlength.
            if len(buffer) == BUFFER_MAX_LENGTH:
                
                # Check if the current hand gesture is 'INDEX POINTING UP'.
                if current_gesture == 'INDEX POINTING UP':
                    
                    # Write 'Draw Mode Enabled' on the frame.
                    cv2.putText(frame, 'Draw Mode Enabled', (5, int(height-20)), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
                    
                    # Get the x and y coordinates of tip of the index finger of the right hand.
                    x, y = hands_tips_positions['RIGHT']['INDEX']
                    
                    # Check if the right hand was detected in the frame.
                    if x and y:
                        
                        # Check if the previous x and y donot have valid values.
                        if not(prev_x) and not(prev_y):
                            
                            # Set the previous x and y to the current x and y values.
                            prev_x, prev_y = x, y
                        
                        # Draw a white line on the frame from previous x and y to the current x and y.
                        cv2.line(frame, (prev_x, prev_y), (x, y), (255,255,255), brush_size)
                        
                        # Create an arbitrary shaped object by attaching different line segments to a body. 
                        drawing_body, drawing_shape, drawing_pts_diff = createObject(frame, drawing_body,
                                                                                     (prev_x, prev_y), (x, y))
                        # Append the shape into the list.
                        shapes.append(drawing_shape)
                        
                        # Append the body and the points of the line segment w.r.t the body position into the list. 
                        to_draw['Objects'].append((drawing_body, drawing_pts_diff))
                        
                        # Update the previous x and y to the current x and y values.
                        prev_x, prev_y = x, y
                
                # Check if the current hand gesture is 'VICTORY'.
                elif current_gesture == 'VICTORY':
                    
                    # Write 'Draw Balls Mode Enabled' on the frame.
                    cv2.putText(frame, 'Draw Balls Mode Enabled', (5, int(height-20)),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
                    
                    # Calculate the ball size accordingly to the right hand size.
                    ball_size = calculateDistance(frame, point1=hands_tips_positions['RIGHT']['PINKY'],
                                                   point2=hands_tips_positions['RIGHT']['THUMB'], draw=False, display=False)
                    
                    # Check if the ball size is calculated.
                    # This will be None in case the right hand was not detected in the frame.
                    if ball_size:

                        # Calculate the radius of the ball to be created.
                        ball_radius = int(ball_size/2)

                        # Create a ball at a random x-coordinate value at the top of the frame.
                        (ball_body, ball_shape), ball_color = createBall(frame, ball_position=(hands_tips_positions['RIGHT']['MIDDLE']), 
                                                                         radius=ball_radius)

                        # Append the ball's body, shape, radius, and color into the list.
                        to_draw['Balls'].append((ball_body, ball_shape, ball_radius, ball_color))

                        # Add both the body and the shape of the ball to the simulation.
                        space.add(ball_body, ball_shape)
                
                # Check if the current hand gesture is 'HIGH-FIVE'.
                elif current_gesture == 'HIGH-FIVE':
                    
                    # Write 'Clear Everything' on the frame.
                    cv2.putText(img=frame, text='Clear Everything', org=(10, height-30), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                                fontScale=1, color=(0,255,0), thickness=2)
                    
                    # Iterate over the bodies in the space.
                    for body in space.bodies:
                        
                        # Check if the body we are iterating upon, is DYNAMIC.
                        if body.body_type == pymunk.Body.DYNAMIC:
                            
                            # Remove the body and the shapes attached to it, from the space.
                            space.remove(body, *list(body.shapes))
                            
                    # Remove all the balls and the objects from the drawing list as well.
                    to_draw['Balls']=[]
                    to_draw['Objects']=[]
    
    # Check if the hands landmarks in the frame are not detected or the current hand gesture is not 'INDEX POINTING UP'.
    if not(results.multi_hand_landmarks) or (len(buffer) == 10 and current_gesture != 'INDEX POINTING UP'):
        
        # Check if the length of shapes list is > 0.
        if len(shapes)>0:
            
            # Add the body and the shapes attached to it, to the space (simulation).
            space.add(drawing_body, *shapes)
            
            # Clear the shapes list.
            shapes = []
        
        # Re-initialize a few variables.
        drawing_body=None
        prev_x = None
        prev_y = None
    
    # Calaculate average frames per second.
    ##################################################################################################
    
    # Get the current time.
    curr_time = time()
    
    # Check if the difference between the start and current time > 0 to avoid division by zero.
    if (curr_time - start_time) > 0:
    
        # Calculate the number of frames per second.
        frames_per_second = frame_count // (curr_time - start_time)
        
        # Write the calculated number of frames per second on the frame. 
        cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, int(width/25)),
                    cv2.FONT_HERSHEY_PLAIN, int(width/300), (0, 255, 0), int(width/200))
    
    ##################################################################################################
            
    # Draw the created balls, the boundary lines and the objects on the frame.
    frame = drawObstacles(frame, to_draw['boundary_lines'])
    frame = drawObjects(frame, to_draw['Objects'], brush_size)
    frame = drawBalls(frame, to_draw['Balls'])
    
    # Check if the FPS is > 0, to avoid division by zero.
    if frames_per_second > 0:
        
        # Step the simulation one step forward.
        space.step(1/frames_per_second)
    
    # Display the frame.
    cv2.imshow("Webcam Feed", frame)
    
    # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF
    
    # Check if 'ESC' is pressed and break the loop.
    if k == 27:
        break    

# Release the VideoCapture Object and close the windows.
camera_video.release()
cv2.destroyAllWindows()



# Additional information:
#           - In summary, this program employed various 
#             functionalities from the previous lessons.
#             Here, we use the hand gestures "Victory",
#             "Hand pointing up", and "High Five" to tell
#             the program which actions we would like to do.
#           - First, when the program starts, we initialize 
#             the camera, and the Pymunk space.
#           - Then, we create the obstacles to prevent the 
#             objects from falling out of the screen.
#           - For the drawing, as long as the user is in drawing
#             mode, the program will record the start and end points
#             of each stroke that was done. Then, after going out of
#             drawing mode, the program will use those to create 
#             line segments that will be attached to the body.

Cool! right? I bet you didn't expect to create something this amazing with a few simple lines of code. Credit goes to Pymunk of course. 

### **<font style="color:rgb(255,140,0)"> Code License Agreement </font>**
```
Copyright (c) 2022 Bleedai.com

Feel free to use this code for your own projects commercial or noncommercial, these projects can be Research-based, just for fun, for-profit, or even Education with the exception that you’re not going to use it for developing a course, book, guide, or any other educational products.

Under *NO CONDITION OR CIRCUMSTANCE* you may use this code for your own paid educational or self-promotional ventures without written consent from Taha Anwar (BleedAI.com).

```